In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
iris = np.loadtxt("../data/iris/iris.tsv")


In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# iris_tsne = TSNE().fit_transform(iris)

In [ ]:
# plt.figure()
# plt.scatter(*iris_tsne.T)
# plt.show()


In [ ]:
iris.shape

In [ ]:
from sklearn.decomposition import PCA

iris_pca = PCA().fit(iris)

In [ ]:
iris_pca.components_

In [ ]:
iris_pca.transform(iris)

In [ ]:
import sys
sys.path.append('../src/')
import tree_reader as tr 
import lumberjack

In [ ]:
forest = lumberjack.fit(iris,trees=100,ifs=3,ofs=3,braids=1,ss=100,leaves=10,depth=4)

In [ ]:
n = forest.nodes()[1]
n_s = n.sister()

sisters = forest.node_representation([n,n_s],mode="sister")

np.dot(sisters,sisters.T)
from tree_reader_utils import sister_distance

sister_distance(sisters)

In [ ]:
# index = 7
# print(forest.nodes()[index].level)
# forest.nodes()[index].mean_residuals()
# forest.split_clusters[2].error_ratio()

# forest.split_clusters[2].nodes[1].mean_residuals()
# forest.split_clusters[2].nodes[1].sister().mean_residuals()


In [ ]:
forest.nodes()[1].mean_error_ratio()

In [ ]:
forest.reset_split_clusters()
forest.interpret_splits(k=100,mode='sister',metric='sister',pca=False,relatives=False)
print(len(forest.split_clusters))
forest.maximum_spanning_tree(mode='samples')
forest.html_tree_summary(n=3)

In [ ]:
forest.split_clusters[1].sisters()

In [ ]:
prediction = forest.predict(iris)
prediction.additive_prediction()


In [ ]:
prediction.prediction_report()

In [ ]:
prediction.jackknife_feature_mse_variance()

In [ ]:
rep = forest.node_sample_encoding(forest.stems())
plt.figure()
# plt.imshow(rep)
plt.imshow(tr.Forest.agglomerate_representation(rep))
plt.show()

In [ ]:
forest.reset_clusters()
forest.cluster_samples_encoding(k=30,metric='cosine')

In [ ]:
forest.tsne(pca=2)
forest.plot_sample_clusters()

In [ ]:
forest.reset_split_clusters()
forest.interpret_splits(mode='additive_mean',k=100,resolution=.1,metric='cosine',relatives=True)
print(len(forest.split_clusters))

In [ ]:
forest.weigh_nodes()
# fig = forest.split_clusters[2].top_local(2)
# fig.savefig('tmp.png',bbox_inches='tight')
# # for i in range(len(forest.split_clusters)):
# #     forest.split_clusters[i].top_local(2,no_plot=True)
forest.maximum_spanning_tree(mode='samples')
# forest.tsne(pca=2)
forest.html_tree_summary(n=2)

In [ ]:
forest.probability_enrichment()

In [ ]:
forest.weighted_node_vector_prediction(forest.roots())

In [ ]:
print(np.mean(iris[:50],axis=0))
print(np.mean(iris[50:100],axis=0))
print(np.mean(iris[100:],axis=0))

In [ ]:
plt.figure()
plt.imshow(pred,aspect='auto',cmap='bwr',vmin=-1,vmax=1,interpolation='none')
plt.show()

In [ ]:
predicted = forest.predict_additive(iris)

In [ ]:
plt.figure()
plt.imshow(predicted.T[:4].T,aspect='auto',interpolation='none')
plt.colorbar()

In [ ]:
plt.figure()
plt.imshow(iris,aspect='auto',interpolation='none')
plt.colorbar()

In [ ]:
forest.split_clusters[1].top_split_features()

In [ ]:
prediction = forest.predict(iris)

In [ ]:
plt.figure()
plt.imshow(prediction.factor_matrix(),aspect='auto',cmap='bwr')
plt.show()

In [ ]:
# f,r1,r2 = forest.split_clusters[1].regression()

In [ ]:
# r2.score(iris.T[f].T,iris)

In [ ]:
forest.sample_clusters[1].logistic_features()

In [ ]:
forest.maximum_spanning_tree(mode='means')

In [ ]:
import community
import networkx as nx

In [ ]:
encoding = forest.node_sample_encoding(forest.leaves())

In [ ]:
plt.figure()
plt.imshow(tr.coocurrence_distance(encoding))
plt.colorbar()
plt.show()

In [ ]:
hacked = tr.hacked_louvain(encoding,k=10,resolution=.5,metric="euclidean")
print(len(set(hacked)))

In [ ]:
forest.reset_sample_clusters()
forest.set_sample_labels(hacked)
print(len(forest.sample_clusters))

In [ ]:
forest.plot_sample_clusters()

In [ ]:
np.log(forest.split_cluster_odds_ratios())

In [ ]:
delta = np.identity(5)

In [ ]:
tiled_indices = np.tile(np.arange(delta.shape[0]),((delta.shape[0]),1))


In [ ]:
tiled_indices

In [ ]:
tiled_indices.T

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.imshow(np.identity(5),vmin=-1,vmax=1)
ax.text(1,1,"hi",c='b')

In [ ]:
type(ax)

In [ ]:
from scipy.spatial.distance import cdist,pdist,squareform

def fast_knn(elements, k, neighborhood_fraction=.01, metric='euclidean'):
    
    # Finds the indices of k nearest neighbors for each sample in a matrix, 
    # using any of the standard scipy distance metrics.

    nearest_neighbors = np.zeros((elements.shape[0], k), dtype=int)
    complete = np.zeros(elements.shape[0], dtype=bool)

    neighborhood_size = max(
        k * 3, int(elements.shape[0] * neighborhood_fraction))
    anchor_loops = 0

    while np.sum(complete) < complete.shape[0]:

        anchor_loops += 1

        available = np.arange(complete.shape[0])[~complete]
        np.random.shuffle(available)
        anchors = available[:int(complete.shape[0] / neighborhood_size) * 3]

        for anchor in anchors:
            print(f"Complete:{np.sum(complete)}\r", end='')

            anchor_distances = cdist(elements[anchor].reshape(
                1, -1), elements, metric=metric)[0]

            neighborhood = np.argpartition(anchor_distances, neighborhood_size)[
                :neighborhood_size]
            anchor_local = np.where(neighborhood == anchor)[0]
            
            local_distances = squareform(
                pdist(elements[neighborhood], metric=metric))

            anchor_to_worst = np.max(local_distances[anchor_local])

            for i, sample in enumerate(neighborhood):
                if not complete[sample]:

                    # First select the indices in the neighborhood that are knn
                    best_neighbors_local = np.argpartition(
                        local_distances[i], k+1)

                    # Next find the worst neighbor among the knn observed
                    best_worst_local = best_neighbors_local[np.argmax(local_distances[i][best_neighbors_local[:k+1]])]
                    # And store the worst distance among the local knn
                    best_worst_distance = local_distances[i,best_worst_local]
                    # Find the distance of the anchor to the central element
                    anchor_distance = local_distances[anchor_local, i]

                    # By the triangle inequality the closest any element outside the neighborhood
                    # can be to element we are examining is the criterion distance:
                    criterion_distance = anchor_to_worst - anchor_distance

#                     if sample == 0:
#                         print(f"ld:{local_distances[i][best_neighbors_local[:k]]}")
#                         print(f"bwd:{best_worst_distance}")
#                         print(f"cd:{criterion_distance}")
                    
                    # Therefore if the criterion distance is greater than the best worst distance, the local knn
                    # is also the best global knn

                    if best_worst_distance >= criterion_distance:
                        continue
                    else:
                        # Before we conclude we must exclude the sample itself from its 
                        # k nearest neighbors
                        best_neighbors_local = [bn for bn in best_neighbors_local[:k+1] if bn !=i]
                        # Finally translate the local best knn to the global indices
                        best_neighbors = neighborhood[best_neighbors_local]

                        nearest_neighbors[sample] = best_neighbors
                        complete[sample] = True
    print("\n")

    return nearest_neighbors



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../src/')
import tree_reader as tr 

iris = np.loadtxt("../data/iris/iris.tsv")

# fast_knn = tr.fast_knn

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor as ETR
import sys
sys.path.append('/Users/bbrener1/battle/localfactorization/')
from binary_forest import BinaryForest

In [ ]:
model = ETR(max_depth=5).fit(iris,iris)

parents = BinaryForest.sklearn_parents(model)

In [ ]:
parents

In [ ]:
encoding = BinaryForest.sklearn_node_sample(iris,model)

In [ ]:
encoding.shape

In [ ]:
from scipy.cluster.hierarchy import linkage,dendrogram

sample_agglomeration = dendrogram(linkage(encoding.T, metric='cosine', method='average'), no_plot=True)['leaves']
node_agglomeration = dendrogram(linkage(encoding, metric='cosine', method='average'), no_plot=True)['leaves']


In [ ]:
plt.figure()
plt.imshow(encoding[node_agglomeration].T[sample_agglomeration].T,aspect='auto')
plt.show()

In [ ]:
BinaryForest.sklearn_parents(model)


In [ ]:
tree = model.estimators_[0].tree_

left_parents = np.ones(tree.node_count) * -1
right_parents = np.ones(tree.node_count) * -1

for node in range(tree.node_count):
    left_child = tree.children_left[node]
    right_child = tree.children_right[node]
    if left_child > 0 and right_child > 0:
        left_parents.append[node]
        right_parents.append[node]
    
    

In [ ]:
sisters = BinaryForest.sklearn_sisters(model)

In [ ]:
for n in range(len(sisters)):
    if sisters[n] != n:
        print(n)
        print(sisters[n])
        print(sisters[sisters[n]])
        print("Error")
        raise Exception()

In [ ]:
np.sum(np.array([1,2,3]).reshape(1,-1),axis=1)[0]